# *ETL realizado sobre los datos de Google*

## Comenzamos con el dataset de las reviews

***importamos librerias a utilizar***

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
import re

***Extraemos los datos***

In [2]:
# Especifica la ruta de la carpeta donde están los archivos JSON
folder_path = r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\reviews_estados'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_reviews_g = pd.concat(dataframes, ignore_index=True)


In [4]:
df_reviews_g.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\reviews_unificado.parquet')

*Se puede iniciar desde aqui con el archivo ya unificado*

In [2]:
df_reviews_g = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\reviews_unificado.parquet')

***Visualizamos la cantidad de registros de este Dataset***

In [3]:
len(df_reviews_g)

2700000

***Transformaciones iniciales***

In [4]:
# Eliminar la columna 'pics' ya que no la necesitamos para este analisis
df_reviews_g.drop(columns=['pics'], inplace=True)

# Convertir los diccionarios en la columna 'resp' a cadenas de texto
df_reviews_g['resp'] = df_reviews_g['resp'].apply(lambda x: str(x) if isinstance(x, dict) else x)

# Convertir la columna 'time' a formato de fecha
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], unit='ms')

# Eliminar duplicados
df_reviews_g.drop_duplicates(inplace=True)

Se realizan las primeras transformaciones del dataset. Se convierten diccionarios a strings, y la columna time a datetime.
Además, se eliminan duplicados y la columna pics (la cual se había determinado que no era necesaria en el análisis preliminar).

***Cambiamos la columna time para obtener de ella tan solo el dia/mes/año de la misma***

In [5]:
# Extraer solo el día/mes/año (formato 'dd/mm/yyyy')
df_reviews_g['time'] = df_reviews_g['time'].dt.strftime('%d/%m/%Y')

**En un analisis de primera instancia, determinamos tambien, que la funcionalidad de hacer una reseña en un local existente en Google Maps se agrego en el año 2007. Por lo tanto todas las reseñas previas a este año las consideramos erroneas y son descartadas.**

In [6]:
# Asegurarse de que la columna 'time' esté en formato datetime
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], errors='coerce')

# Detectar outliers en 'time' (fechas fuera del rango)
min_date = pd.to_datetime('2007-06-01')
max_date = pd.to_datetime('today')

# Filtrar las filas que tienen fechas fuera de este rango
outliers_time = df_reviews_g[(df_reviews_g['time'] < min_date) | (df_reviews_g['time'] > max_date)]

# Eliminamos estos outliers
df_reviews_g = df_reviews_g[(df_reviews_g['time'] >= min_date) & (df_reviews_g['time'] <= max_date)]

***Eliminamos la columna 'resp' ya que no la necesitamos para este analisis***

In [7]:
df_reviews_g = df_reviews_g.drop(columns=['resp'])

***Reemplazamos valores nulos en la columna 'review_text' con 'Sin Reseña'***

In [8]:
df_reviews_g['text'] = df_reviews_g['text'].fillna('Sin Reseña')

***Luego de estas transformaciones, revisamos nuevamente la cantidad de registros***

In [9]:
len(df_reviews_g)

1046874

In [10]:
df_reviews_g.head(3)

,user_id,name,time,rating,text,gmap_id
0,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-08-03,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49


## Pasamos al dataset sobre los Metadatos

***Extraemos los datos***

In [13]:
# Ruta del archivo .pkl
folder_path = r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\metadata-sitios\metadata-sitios'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_business_google = pd.concat(dataframes, ignore_index=True)

In [14]:
df_business_google.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\metadatos_unificado.parquet')

**Se puede iniciar desde aquí con el dataset ya generado**

In [18]:
df_business_google = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\metadatos_unificado.parquet')

In [12]:
df_business_google.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.29213,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


***Verificamos la cantidad de registros***

In [11]:
len(df_business_google)

3025011

***Desanidamos Misc***

In [19]:
# Desanidar la columna 'MISC' que contiene JSON
misc_expanded = pd.json_normalize(df_business_google['MISC'])

# Combinar las columnas desanidadas con el DataFrame original
df_business_google = pd.concat([df_business_google, misc_expanded], axis=1)

# Eliminar la columna original 'MISC'
df_business_google = df_business_google.drop(columns=['MISC'])

# Convertir listas o arrays en cadenas separadas por comas en todas las columnas
df_business_google = df_business_google.apply(
    lambda col: col.apply(lambda x: ', '.join(map(str, x)) if isinstance(x, (list, np.ndarray)) else x)
)

***Hacemos transformaciones***

In [20]:
# Listamos las columnas no deseadas
col_eliminar = ['price', 'hours', 'url', 'Health & safety', 'Accessibility', 'Planning', 'description',
        'Offerings', 'Amenities', 'Atmosphere', 'Payments', 'Popular for',
        'Dining options', 'Crowd', 'From the business', 'Highlights',
        'Recycling', 'Getting here', 'Activities', 'Lodging options',
        'Health and safety']

# Eliminamos las columnas no deseadas
df_business_google = df_business_google.drop(columns=col_eliminar)

# Eliminar filas duplicadas completamente idénticas
df_business_google = df_business_google.drop_duplicates()

# Redondear las columnas 'latitude' y 'longitude' a 7 decimales
df_business_google['latitude'] = df_business_google['latitude'].round(7)
df_business_google['longitude'] = df_business_google['longitude'].round(7)

# Convertir la columna 'state' a tipo texto (string)
df_business_google['state'] = df_business_google['state'].astype(str)

# Eliminar filas donde 'state' es igual a 'Permanently closed'
df_business_google = df_business_google[df_business_google['state'] != 'Permanently closed']

# Rellenamos los valores nulos con el texto elegido
df_business_google['category'] = df_business_google['category'].fillna('Valores Faltantes')
df_business_google['relative_results'] = df_business_google['relative_results'].fillna('Sin resultados vinculados')
df_business_google['Service options'] = df_business_google['Service options'].fillna('Sin dato')

***Verificamos la cantidad de registros luego de los cambios realizados***

In [21]:
len(df_business_google)

2809262

***Filtramos para dejar solo los registros con categorias que consideramos dentro de la gastronomia***

In [22]:
# Definimos palabras clave a buscar luego en las categorias
palabra = r'\b(restaurant|cafe|delivery|diner|bistro|takeout|bar|pub|grill|pizzeria|coffee|bakery|food|eatery|sandwich|snack)\b'

# Filtrar las categorías coincidentes
df_business_google = df_business_google[df_business_google['category'].str.contains(palabra, case=False, regex=True, na=False)]

C:\Users\GASTON\AppData\Local\Temp\ipykernel_6180\3655407268.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_business_google = df_business_google[df_business_google['category'].str.contains(palabra, case=False, regex=True, na=False)]


***Generamos una columna "clasificacion" a partir de la categoria***

In [23]:
clasificacion = {
    'Dining Venue': ['Restaurant'],
    'Quick Service': ['Fast food restaurant', 'Pizza restaurant', 'Pizza Takeout'],
    'Takeout & Delivery': ['Takeout Restaurant', 'Delivery Restaurant'],
    'Mexican Dining': ['Mexican restaurant', 'Taco restaurant'],
    'American Dining': ['American restaurant', 'Burger restaurant'],
    'Sandwich Bar': ['Sandwich shop'],
    'Italian Dining': ['Italian restaurant'],
    'Chinese Dining': ['Chinese restaurant'],
    'Café & Coffee': ['Coffee shop', 'Cafe'],
    'Seafood Dining': ['Seafood restaurant'],
    'Barbecue Dining': ['Barbecue restaurant'],
    'Asian Fusion': ['Asian restaurant', 'Sushi restaurant', 'Japanese restaurant', 'Thai restaurant'],
    'Chicken House': ['Chicken restaurant', 'Chicken wings restaurant'],
    'Bakery Shop': ['Bakery', 'Bakery shop'],
    'Ice Cream Parlor': ['Ice cream shop'],
    'Indian Dining': ['Indian restaurant'],
    'Latin American Dining': ['Latin American restaurant'],
    'Juice Bar': ['Juice shop'],
    'Specialty Catering': ['Caterer'],
    'Vegetarian Dining': ['Vegetarian restaurant'],
    'Vietnamese Dining': ['Vietnamese Restaurant'],
    'Health Foods': ['Health food store', 'Health Food', 'Green Food']
}


def assign_group(category):
    """
    
    """
    if pd.isna(category):  # Si la categoría es Nula
        return 'Other'
    for key, values in clasificacion.items():
        for value in values:
            if value.lower() in category.lower():
                return key
    return 'Other'

df_business_google['clasificacion'] = df_business_google['category'].map(assign_group)

In [24]:
df_business_google = df_business_google[df_business_google['clasificacion'] != 'Other']

In [25]:
df_business_google.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state,relative_results,Service options,clasificacion
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,Open ⋅ Closes 6PM,"0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2c...","Takeout, Dine-in, Delivery",Dining Venue
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,Restaurant,4.5,18,Open ⋅ Closes 9:30PM,Sin resultados vinculados,"Outdoor seating, Curbside pickup, No-contact d...",Dining Venue
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,Restaurant,4.0,8,None,"0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6d...",Delivery,Dining Venue


### Filtramos los datos mediante el uso de la columna gmap_id (compartida por ambos datasets) para asi dejar tan solo los datos de california en el Dataframe con los datos de los locales.

In [26]:
# Filtrar los registros en df_business_google que coincidan en la columna 'gmap_id'
df_business_google = df_business_google[df_business_google['gmap_id'].isin(df_reviews_g['gmap_id'])]

***Verificamos nuevamente la cantidad de registros para comparar***

In [27]:
len(df_business_google)

7918

### Realizamos Ultimas transformaciones al Dataset de business de Google

***Eliminamos los valores nulos de address***

In [28]:
# Eliminar las filas que tienen valores nulos en la columna 'address'
df_business_google = df_business_google.dropna(subset=['address'])

***Este codigo extrae la ciudad y direccion en columnas separadas despues de considerar los posibles problemas de carga, se eliminaran valores nulos que queden en el dataframe luego de este proceso***

In [29]:
# Función para extraer dirección y ciudad de la columna 'address'
def extract_address_city(row):
    # Verificar si el valor en 'address' no es nulo
    if pd.isnull(row['address']):
        return pd.Series([None, None])
    
    # Verificar si el nombre contiene una coma
    if ',' in row['name']:
        # Si hay coma en el nombre, ajustamos el patrón para excluir el nombre del análisis
        # Permitimos caracteres especiales en la dirección y ciudad
        address_pattern = r'(?P<address>.*?),\s*(?P<city>[A-Za-z\s\'•\-]+),\s*(?P<state_zip>[A-Z]{2}\s\d{5})'
        match = re.search(address_pattern, row['address'])
    else:
        # Usamos el patrón completo con nombre
        # Permitimos caracteres especiales en el nombre, dirección y ciudad
        address_pattern = r'(?P<name>.*?),\s*(?P<address>.*?),\s*(?P<city>[A-Za-z\s\'•\-]+),\s*(?P<state_zip>[A-Z]{2}\s\d{5})'
        match = re.search(address_pattern, row['address'])

    if match:
        # Verificar si la dirección capturada contiene "port" (como en "Airport")
        if 'port' in match.group('address').lower():
            address = match.group('address').strip()
            city = match.group('city').strip()
            return pd.Series([address, city])
        
        # Verificar si el nombre coincide con la columna 'name' (solo si se usó el patrón con nombre)
        if 'name' in match.groupdict() and row['name'].strip() == match.group('name').strip():
            address = match.group('address').strip()
            city = match.group('city').strip()
            return pd.Series([address, city])
        elif 'name' not in match.groupdict():
            # Si no estamos utilizando el nombre, simplemente tomamos la dirección y la ciudad
            address = match.group('address').strip()
            city = match.group('city').strip()
            return pd.Series([address, city])

    # Si no coincide o no encuentra suficientes datos
    return pd.Series([None, None])

# Aplicar la función a cada fila del DataFrame y guardar los resultados en 'address_depurada' y 'city'
df_business_google[['address_depurada', 'city']] = df_business_google.apply(extract_address_city, axis=1)

In [22]:
df_business_google.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state,relative_results,Service options,clasificacion,address_depurada,city
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,Open ⋅ Closes 6PM,"0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2c...","Takeout, Dine-in, Delivery",Dining Venue,761 S Vermont Ave,Los Angeles
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,Restaurant,4.5,18,Open ⋅ Closes 9:30PM,Sin resultados vinculados,"Outdoor seating, Curbside pickup, No-contact d...",Dining Venue,12740 La Mirada Blvd,La Mirada
761,La Potranca,"La Potranca, 12821 Venice Blvd., Los Angeles, ...",0x80c2baf50d29bf63:0x5bd904b842b9fcc,34.000181,-118.441249,Restaurant,4.2,13,Closed ⋅ Opens 10AM,"0x80c2bac345536273:0x8b015c3512788465, 0x80c2c...","Dine-in, Delivery",Dining Venue,12821 Venice Blvd.,Los Angeles
1426,Circle K,"Circle K, 16331 Lakeshore Dr, Lake Elsinore, C...",0x80dc976f028eb61d:0x1a5ed32889a67122,33.689862,-117.376151,"Convenience store, ATM, Coffee shop, Coffee st...",3.5,24,Open 24 hours,"0x80dc99d523c8983b:0x5bcf75a72567785e, 0x80dc9...","In-store shopping, Delivery",Café & Coffee,16331 Lakeshore Dr,Lake Elsinore
1548,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,"Hamburger restaurant, American restaurant, Bar...",3.7,38,Closed ⋅ Opens 6AM,"0x80c2d765f8dd4ebf:0xb6baf31e3e536ffa, 0x80c2d...","Drive-through, Takeout, Dine-in, Delivery",Dining Venue,13101 Ramona Blvd,Baldwin Park


### Procedemos a realizar una columna "county" que detalle el condado del registro, utilizando la columna "city" para ubicarnos

***Limpiamos la columna de referencia***

In [30]:
# Eliminamos Nulos de la columna 'city'
df_business_google = df_business_google.dropna(subset=['city'])

# Limpiamos espacios en la misma
df_business_google['city'] = df_business_google['city'].str.strip()

***Creamos diccionario extenso con los condados y ciudades que abarca***

In [31]:
condados_a_ciudades = {
    'Alameda': ['Alameda', 'Albany', 'Berkeley', 'Dublin', 'Emeryville', 'Fremont', 'Hayward', 'Livermore', 'Newark', 'Oakland', 'Pleasanton', 'San Leandro', 'Union City', 'Castro Valley', 'San Lorenzo'],
    'Alpine': ['Markleeville', 'Markleeville'],
    'Amador': ['Amador City', 'Ione', 'Jackson', 'Plymouth', 'Sutter Creek', 'Jackson'],
    'Butte': ['Biggs', 'Chico', 'Gridley', 'Oroville', 'Durham', 'Paradise', 'Chico', 'Magalia', 'Paradise'],
    'Calaveras': ['Angels Camp', 'Copperopolis', 'Arnold', 'San Andreas'],
    'Colusa': ['Colusa', 'Williams', 'Colusa', 'Princeton'],
    'Contra Costa': ['Alameda Park', 'Antioch', 'Brentwood', 'Clayton', 'Concord', 'Crockett', 'Danville', 'El Cerrito', 'Hercules', 'Lafayette', 'Martinez', 'Moraga', 'Orinda', 'Pinole', 'Pittsburg', 'Pleasant Hill', 'Richmond', 'San Pablo', 'San Ramon', 'Walnut Creek', 'Alamo', 'Pacheco', 'Bay Point', 'El Sobrante', 'Oakley', 'Rodeo', 'Discovery Bay'],
    'Del Norte': ['Crescent City'],
    'El Dorado': ['Placerville', 'South Lake Tahoe', 'El Dorado Hills', 'Cool', 'Lotus', 'Cameron Park', 'Kyburz', 'Diamond Springs'],
    'Fresno': ['Biola', 'Clovis', 'Coalinga', 'Firebaugh', 'Fowler', 'Fresno', 'Huron', 'Kerman', 'Kingsburg', 'Mendota', 'Orange Cove', 'Parlier', 'Reedley', 'San Joaquin', 'Sanger', 'Selma', 'Prather', 'Shaver Lake', 'Riverdale', 'Five Points'],
    'Glenn': ['Orland', 'Willows'],
    'Humboldt': ['Arcata', 'Blue Lake', 'Eureka', 'Fortuna', 'Rio Dell', 'Trinidad', 'McKinleyville', 'Ferndale', 'Garberville', 'Loleta', 'Whitethorn', 'Willow Creek'],
    'Imperial': ['Brawley', 'Calexico', 'Calipatria', 'El Centro', 'Holtville', 'Imperial', 'Westmorland'],
    'Inyo': ['Bishop', 'Lone Pine'],
    'Kern': ['Arvin', 'Bakersfield', 'California City', 'Delano', 'Maricopa', 'McFarland', 'Ridgecrest', 'Shafter', 'Taft', 'Tehachapi', 'Wasco', 'Rosamond', 'Edwards AFB', 'Lamont', 'Mojave', 'Lost Hills', 'Kernville', 'Lake Isabella', 'Edwards'],
    'Kings': ['Avenal', 'Corcoran', 'Hanford', 'Lemoore'],
    'Lake': ['Clearlake', 'Lakeport', 'Nice', 'Spring Valley', 'Hidden Valley Lake', 'Kelseyville', 'Lower Lake'],
    'Lassen': ['Susanville', 'Bieber'],
    'Los Angeles': ['Pacific Palisades', 'Valencia', 'Reseda', 'San Pedro', 'City of Industry', 'Northridge', 'North Hollywood', 'Sylmar', 'Sun Valley', 'Artesia', 'Arcadia', 'Venice', 'Sherman Oaks', 'Van Nuys', 'Studio City', 'Granada Hills', 'Rowland Heights', 'Harbor City', 'Tujunga', 'Saugus', 'Highland Park', 'Pacoima', 'Universal City', 'Hollywood', 'East Los Angeles', 'West Hills', 'Cudahy', 'La Crescenta-Montrose', 'Canoga Park','Wilmington', 'Woodland Hills', 'Canyon Country', 'Panorama City', 'Tarzana', 'Winnetka', 'Newhall', 'Castaic', 'Stevenson Ranch', 'Irwindale', 'View Park-Windsor Hills', 'Lomita', 'Marina Del Rey', 'Valley Glen', 'Eagle Rock', 'North Hills', 'Lakewood', 'Arleta', 'Toluca Lake', 'Valley Village', 'Mission Hills', 'Bassett', 'Littlerock', 'Topanga', 'Playa Vista', 'Quartz Hill', 'East Compton', 'Porter Ranch', 'Silver Lake'],
    'Acton': ['Agoura Hills', 'Alhambra', 'Alondra Park', 'Altadena', 'Avalon', 'Azusa', 'Baldwin Park', 'Bell', 'Bell Gardens', 'Bellflower', 'Beverly Hills', 'Burbank', 'Calabasas', 'Carson', 'Cerritos', 'Chatsworth', 'Claremont', 'Commerce', 'Compton', 'Covina', 'Culver City', 'Diamond Bar', 'Downey', 'Duarte', 'El Monte', 'El Segundo', 'Encino', 'Gardena', 'Glendale', 'Glendora', 'Hacienda Heights', 'Hawaiian Gardens', 'Hawthorne', 'Hermosa Beach', 'Huntington Park', 'Industry', 'Inglewood', 'La Canada Flintridge', 'La Mirada', 'La Puente', 'La Verne', 'Lancaster', 'Lawndale', 'Lennox', 'Long Beach', 'Los Angeles', 'Lynwood', 'Malibu', 'Manhattan Beach', 'Maywood', 'Monrovia', 'Montebello', 'Monterey Park', 'Norwalk', 'Palmdale', 'Palos Verdes Estates', 'Paramount', 'Pasadena', 'Pico Rivera', 'Pomona', 'Rancho Palos Verdes', 'Redondo Beach', 'Rolling Hills Estates', 'Rosemead', 'San Dimas', 'San Fernando', 'San Gabriel', 'San Marino', 'Santa Clarita', 'Santa Fe Springs', 'Santa Monica', 'Sierra Madre', 'Signal Hill', 'South El Monte', 'South Gate', 'South Pasadena', 'Temple City', 'Torrance', 'Vernon', 'Walnut', 'West Covina', 'West Hollywood', 'Westlake Village', 'Whittier'],
    'Madera': ['Chowchilla', 'Madera', 'Bass Lake', 'North Fork', 'Coarsegold', 'Oakhurst'],
    'Marin': ['Belvedere', 'Corte Madera', 'Fairfax', 'Larkspur', 'Mill Valley', 'Novato', 'Ross', 'San Anselmo', 'San Rafael', 'Sausalito', 'Tiburon', 'Olema', 'Point Reyes Station', 'Greenbrae', 'Stinson Beach', 'Tomales'],
    'Mariposa': ['Mariposa'],
    'Mendocino': ['Fort Bragg', 'Point Arena', 'Ukiah', 'Willits', 'Mendocino', 'Little River', 'Laytonville', 'Philo', 'Gualala', 'Redwood Valley', 'Elk', 'Boonville'],
    'Merced': ['Atwater', 'Dos Palos', 'Gustine', 'Livingston', 'Los Banos', 'Merced', 'Delhi', 'Santa Nella'],
    'Modoc': ['Alturas'],
    'Mono': ['Mammoth Lakes', 'June Lake'],
    'Monterey': ['Carmel-by-the-Sea', 'Del Rey Oaks', 'Gonzales', 'Greenfield', 'King City', 'Marina', 'Monterey', 'Pacific Grove', 'Salinas', 'Sand City', 'Seaside', 'Soledad', 'Carmel Valley', 'Pebble Beach', 'Castroville', 'Pajaro', 'Royal Oaks', 'Prunedale', 'Aromas','Carmel-By-The-Sea'],
    'Napa': ['American Canyon', 'Calistoga', 'Napa', 'St. Helena', 'Yountville', 'St Helena', 'Rutherford', 'Santa Elena'],
    'Nevada': ['Grass Valley', 'Nevada City', 'Truckee', 'Cisco Grove', 'North San Juan', 'Penn Valley'],
    'Orange': ['Aliso Viejo', 'Anaheim', 'Brea', 'Buena Park', 'Costa Mesa', 'Cypress', 'Dana Point', 'Fountain Valley', 'Fullerton', 'Garden Grove', 'Huntington Beach', 'Irvine', 'La Habra', 'La Palma', 'Laguna Beach', 'Laguna Hills', 'Laguna Niguel', 'Laguna Woods', 'Lake Forest', 'Los Alamitos', 'Mission Viejo', 'Newport Beach', 'Orange', 'Placentia', 'Rancho Santa Margarita', 'San Clemente', 'San Juan Capistrano', 'Santa Ana', 'Seal Beach', 'Stanton', 'Tustin', 'Villa Park', 'Westminster', 'Yorba Linda', 'Foothill Ranch', 'Ladera Ranch', 'Silverado', 'Trabuco Canyon', 'Newport Coast'],
    'Placer': ['Auburn', 'Colfax', 'Lincoln', 'Loomis', 'Rocklin', 'Roseville', 'Kings Beach', 'Tahoe City', 'Granite Bay', 'Olympic Valley', 'Newcastle', 'Homewood'],
    'Plumas': ['Portola', 'Quincy', 'Chester', 'Lake Almanor', 'Blairsden-Graeagle'],
    'Riverside': ['Banning', 'Beaumont', 'Blythe', 'Calimesa', 'Canyon Lake', 'Cathedral City', 'Coachella', 'Corona', 'Desert Hot Springs', 'Eastvale', 'Hemet', 'Indian Wells', 'Indio', 'Jurupa Valley', 'La Quinta', 'Lake Elsinore', 'Menifee', 'Moreno Valley', 'Murrieta', 'Norco', 'Palm Desert', 'Palm Springs', 'Perris', 'Rancho Mirage', 'Riverside', 'San Jacinto', 'Temecula', 'Wildomar', 'Thousand Palms', 'Cabazon', 'March Air Reserve Base', 'Idyllwild-Pine Cove', 'Thermal', 'Bermuda Dunes', 'Mira Loma', 'Winchester', 'Anza', 'Mecca', 'Romoland', 'Sun City'],
    'Sacramento': ['Citrus Heights', 'Elk Grove', 'Folsom', 'Galt', 'Isleton', 'Rancho Cordova', 'Sacramento', 'Antelope', 'Arden-Arcade', 'Carmichael', 'Clay', 'Courtland', 'Elverta', 'Fair Oaks', 'Florin', 'Foothill Farms', 'Franklin', 'Fruitridge Pocket', 'Gold River', 'La Riviera', 'Lemon Hill', 'Mather', 'North Highlands', 'Orangevale', 'Parkway', 'Rancho Murieta', 'Rio Linda', 'Rosemont', 'Vineyard', 'Walnut Grove', 'Wilton', 'Hood', 'Rio Linda'],
    'San Benito': ['Hollister', 'San Juan Bautista', 'Tres Pinos'],
    'San Bernardino': ['Adelanto', 'Apple Valley', 'Barstow', 'Big Bear Lake', 'Chino', 'Chino Hills', 'Colton', 'Fontana', 'Grand Terrace', 'Hesperia', 'Highland', 'Loma Linda', 'Montclair', 'Needles', 'Ontario', 'Rancho Cucamonga', 'Redlands', 'Rialto', 'San Bernardino', 'Twentynine Palms', 'Upland', 'Victorville', 'Yucaipa', 'Yucca Valley', 'Joshua Tree', 'Crestline', 'Lake Arrowhead', 'Alta Loma', 'Joshua Tree', 'Oro Grande', 'Yermo', 'Bloomington','Phelan','Helendale','Fort Irwin','Running Springs'],
    'San Diego': ['Carlsbad', 'Chula Vista', 'Coronado', 'Del Mar', 'El Cajon', 'Encinitas', 'Escondido', 'Imperial Beach', 'La Mesa', 'Lemon Grove', 'National City', 'Oceanside', 'Poway', 'San Diego', 'San Marcos', 'Santee', 'Solana Beach', 'Vista', 'Alpine', 'Bonita', 'Borrego Springs', 'Fallbrook', 'Jamul', 'Julian', 'Lakeside', 'Pine Valley', 'Ramona', 'Ranchita', 'Rancho Santa Fe', 'Spring Valley', 'Camp Pendleton North', 'Bonsall', 'Pala', 'Carmel Mountain Ranch', 'Pauma Valley', 'Julian','La Jolla','Camp Pendleton'],
    'San Francisco': ['San Francisco'],
    'San Joaquin': ['Escalon', 'Lathrop', 'Lodi', 'Manteca', 'Ripon', 'Stockton', 'Tracy', 'Acampo', 'French Camp', 'Lockeford', 'Thornton', 'Lockeford', 'Acampo'],
    'San Luis Obispo': ['Arroyo Grande', 'Atascadero', 'Grover Beach', 'Morro Bay', 'Paso Robles', 'Pismo Beach', 'San Luis Obispo', 'Baywood-Los Osos', 'Santa Margarita', 'Cayucos', 'Oceano', 'Templeton', 'Nipomo', 'Cambria', 'San Simeon', 'Los Osos'],
    'San Mateo': ['Atherton', 'Belmont', 'Brisbane', 'Burlingame', 'Colma', 'Daly City', 'East Palo Alto', 'Foster City', 'Half Moon Bay', 'Hillsborough', 'Menlo Park', 'Millbrae', 'Pacifica', 'Portola Valley', 'Redwood City', 'San Bruno', 'San Carlos', 'San Mateo', 'South San Francisco', 'Woodside'],
    'Santa Barbara': ['Buellton', 'Carpinteria', 'Goleta', 'Guadalupe', 'Lompoc', 'Santa Barbara', 'Santa Maria', 'Solvang', 'Orcutt', 'Los Alamos','Los Olivos','Montecito','Isla Vista','Vandenberg AFB'],
    'Santa Clara': ['Campbell', 'Cupertino', 'Gilroy', 'Los Altos', 'Los Altos Hills', 'Los Gatos', 'Milpitas', 'Monte Sereno', 'Morgan Hill', 'Mountain View', 'Palo Alto', 'San Jose', 'Santa Clara', 'Saratoga', 'Sunnyvale','Stanford'],
    'Santa Cruz': ['Capitola', 'Santa Cruz', 'Scotts Valley', 'Watsonville', 'Boulder Creek', 'Freedom','Aptos','Soquel','Felton'],
    'Shasta': ['Anderson', 'Redding', 'Shasta Lake', 'Burney', 'McArthur', 'Platina', 'Lakehead', 'Fall River Mills', 'Cottonwood'],
    'Sierra': ['Loyalton', 'Sierra City'],
    'Siskiyou': ['Dorris', 'Dunsmuir', 'Etna', 'Fort Jones', 'Montague', 'Mount Shasta', 'Tulelake', 'Weed', 'Yreka', 'Mt Shasta', 'Mount Shasta'],
    'Solano': ['Benicia', 'Dixon', 'Fairfield', 'Rio Vista', 'Suisun City', 'Vacaville', 'Vallejo'],
    'Sonoma': ['Cloverdale', 'Cotati', 'Healdsburg', 'Petaluma', 'Rohnert Park', 'Santa Rosa', 'Sebastopol', 'Sonoma', 'Windsor', 'Bodega Bay', 'Guerneville', 'Geyserville', 'Forestville', 'Monte Rio'],
    'Stanislaus': ['Ceres', 'Hughson', 'Modesto', 'Newman', 'Oakdale', 'Patterson', 'Riverbank', 'Turlock', 'Waterford', 'Salida', 'Keyes', 'Westley'],
    'Sutter': ['Live Oak', 'Yuba City', 'Sutter'],
    'Tehama': ['Corning', 'Red Bluff', 'Tehama'],
    'Trinity': ['Weaverville', 'Wildwood', 'Lewiston'],
    'Tulare': ['Dinuba', 'Exeter', 'Farmersville', 'Lindsay', 'Porterville', 'Tulare', 'Visalia', 'Woodlake', 'Alpaugh', 'Orosi', 'Springville','Earlimart','Pixley','Terra Bella','Ivanhoe','Strathmore','Richgrove'],
    'Tuolumne': ['Sonora', 'Jamestown', 'Columbia', 'Pinecrest', 'Groveland'],
    'Ventura': ['Camarillo', 'Fillmore', 'Moorpark', 'Ojai', 'Oxnard', 'Port Hueneme', 'Santa Paula', 'Simi Valley', 'Thousand Oaks', 'Ventura', 'Oak Park','Newbury Park','Somis'],
    'Yolo': ['Davis', 'West Sacramento', 'Winters', 'Woodland', 'Zamora'],
    'Yuba': ['Marysville', 'Wheatland','Linda']
}

***Tomando como referencia este diccionario, creamos la columna county***

In [32]:
# Función para asignar el condado basado en la ciudad
def asignar_condado(city):
    for county, cities in condados_a_ciudades.items():
        if city in cities:
            return county
    return None  # Devuelve None si la ciudad no está en ninguna lista

# Crear la nueva columna 'county' aplicando la función a la columna 'city'
df_business_google['county'] = df_business_google['city'].apply(asignar_condado)

In [26]:
df_business_google.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state,relative_results,Service options,clasificacion,address_depurada,city,county
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,Open ⋅ Closes 6PM,"0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2c...","Takeout, Dine-in, Delivery",Dining Venue,761 S Vermont Ave,Los Angeles,Acton
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,Restaurant,4.5,18,Open ⋅ Closes 9:30PM,Sin resultados vinculados,"Outdoor seating, Curbside pickup, No-contact d...",Dining Venue,12740 La Mirada Blvd,La Mirada,Acton
761,La Potranca,"La Potranca, 12821 Venice Blvd., Los Angeles, ...",0x80c2baf50d29bf63:0x5bd904b842b9fcc,34.000181,-118.441249,Restaurant,4.2,13,Closed ⋅ Opens 10AM,"0x80c2bac345536273:0x8b015c3512788465, 0x80c2c...","Dine-in, Delivery",Dining Venue,12821 Venice Blvd.,Los Angeles,Acton
1426,Circle K,"Circle K, 16331 Lakeshore Dr, Lake Elsinore, C...",0x80dc976f028eb61d:0x1a5ed32889a67122,33.689862,-117.376151,"Convenience store, ATM, Coffee shop, Coffee st...",3.5,24,Open 24 hours,"0x80dc99d523c8983b:0x5bcf75a72567785e, 0x80dc9...","In-store shopping, Delivery",Café & Coffee,16331 Lakeshore Dr,Lake Elsinore,Riverside
1548,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,34.079995,-117.988951,"Hamburger restaurant, American restaurant, Bar...",3.7,38,Closed ⋅ Opens 6AM,"0x80c2d765f8dd4ebf:0xb6baf31e3e536ffa, 0x80c2d...","Drive-through, Takeout, Dine-in, Delivery",Dining Venue,13101 Ramona Blvd,Baldwin Park,Acton


### Ahora seleccionamos columnas de interes para el proyecto

In [33]:
df_business_google.columns

Index(['name', 'address', 'gmap_id', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'state', 'relative_results',
       'Service options', 'clasificacion', 'address_depurada', 'city',
       'county'],
      dtype='object')

In [34]:
#Movemos la columna 'gmap_id' a la primera posición:

# Reordenar las columnas colocando 'gmap_id' en la primera posición
cols = ['gmap_id'] + [col for col in df_business_google.columns if col != 'gmap_id']
df_business_google = df_business_google.reindex(columns=cols)

# Verificar que la columna 'gmap_id' está en la primera posición
print(df_business_google.columns)


Index(['gmap_id', 'name', 'address', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'state', 'relative_results',
       'Service options', 'clasificacion', 'address_depurada', 'city',
       'county'],
      dtype='object')


### Formaremos otro dataset que tendra los datos sobre la posición geografica de los locales y se vinculara a los mismos mediante "gmap_id"

In [40]:
df_ubicacion = df_business_google[['gmap_id', 'address_depurada', 'city', 'county', 'latitude', 'longitude']].copy()
# Restablecer el índice para que sea consecutivo
df_ubicacion.reset_index(drop=True, inplace=True)

In [41]:
# Creamos la nueva columna 'id_locacion' con números incrementales, que serán la PK
df_ubicacion['id_locacion'] = range(1, len(df_ubicacion) + 1)

# Reordenar las columnas para que 'id_locacion' esté en primer lugar
df_ubicacion = df_ubicacion[['id_locacion', 'gmap_id', 'address_depurada', 'city', 'county', 'latitude', 'longitude']]

In [42]:
df_ubicacion.head()

,id_locacion,gmap_id,address_depurada,city,county,latitude,longitude
0,1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,761 S Vermont Ave,Los Angeles,Acton,34.058092,-118.292130
1,2,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,12740 La Mirada Blvd,La Mirada,Acton,33.916402,-118.010855
2,3,0x80c2baf50d29bf63:0x5bd904b842b9fcc,12821 Venice Blvd.,Los Angeles,Acton,34.000181,-118.441249
3,4,0x80dc976f028eb61d:0x1a5ed32889a67122,16331 Lakeshore Dr,Lake Elsinore,Riverside,33.689862,-117.376151
4,5,0x80c2d765f8c90a3d:0x16afb75943e7ad50,13101 Ramona Blvd,Baldwin Park,Acton,34.079995,-117.988951


In [43]:
df_business_google.head(3)

,gmap_id,name,address,latitude,longitude,category,avg_rating,num_of_reviews,state,relative_results,Service options,clasificacion,address_depurada,city,county
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",34.058092,-118.292130,Korean restaurant,4.4,18,Open ⋅ Closes 6PM,"0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2c...","Takeout, Dine-in, Delivery",Dining Venue,761 S Vermont Ave,Los Angeles,Acton
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",33.916402,-118.010855,Restaurant,4.5,18,Open ⋅ Closes 9:30PM,Sin resultados vinculados,"Outdoor seating, Curbside pickup, No-contact d...",Dining Venue,12740 La Mirada Blvd,La Mirada,Acton
761,0x80c2baf50d29bf63:0x5bd904b842b9fcc,La Potranca,"La Potranca, 12821 Venice Blvd., Los Angeles, ...",34.000181,-118.441249,Restaurant,4.2,13,Closed ⋅ Opens 10AM,"0x80c2bac345536273:0x8b015c3512788465, 0x80c2c...","Dine-in, Delivery",Dining Venue,12821 Venice Blvd.,Los Angeles,Acton


### Volvemos a transformar el dataset "Reviews"

A partir del dataframe df_reviews_g sacamos la columna gmap_id con valores únicos. De esa forma, esa lista permitió filtrar los registros pertenecientes al Estado de California en el dataset metadatos de Google.
Una vez que tenemos aquellos registros definidos, en los que se ha delimitado el dataset al rubro gastronómico y afines, corresponde ahora volver a filtrar este dataset para que sólo queden a disposición los registros del Estado de california vinculados al sector gastronómico, lo que haremos a continuación.

In [44]:
# Filtrar los registros en df_reviews_g que coincidan en la columna 'gmap_id'
df_reviews_google = df_reviews_g[df_reviews_g['gmap_id'].isin(df_business_google['gmap_id'])]

In [45]:
df_reviews_google.head()

,user_id,name,time,rating,text,gmap_id
0,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-08-03,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-07-03,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
9,1.174299e+20,Y.M,2018-10-06,5,Sin Reseña,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [46]:
df_reviews_google.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'gmap_id'], dtype='object')

In [47]:
#A continuación crearemos una nueva columna 'review_id' para usar como primary key.

# Crear una copia del DataFrame para evitar el SettingWithCopyWarning
df_reviews_google = df_reviews_google.copy()

# Crear la nueva columna 'review_id' con números incrementales
df_reviews_google['review_id'] = range(1, len(df_reviews_google) + 1)

# Reordenar las columnas para que 'review_id' esté en primer lugar
df_reviews_google = df_reviews_google[['review_id', 'user_id', 'name', 'time', 'rating', 'text', 'gmap_id']]

In [48]:
df_reviews_google.head(2)

,review_id,user_id,name,time,rating,text,gmap_id
0,1,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,2,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49


## Guardamos los Dataframes finales en archivos .parquet

In [49]:
df_reviews_google.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reviews_google.parquet')
df_business_google.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\locales_google.parquet')
df_ubicacion.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\ubicacion_google.parquet')